In [1]:
import numpy as np #conda install -c anaconda numpy
import pandas as pd #conda install -c anaconda pandas
import seaborn as sns #conda install -c anaconda seaborn
from sqlalchemy import create_engine #conda install -c anaconda sqlalchemy
from sklearn.metrics.pairwise import cosine_similarity #conda install -c anaconda scikit-learn
from numpy import dot
from numpy.linalg import norm

In [2]:
host = 'localhost'  # localhost本地端
user = 'root'
password = ''
db_name = 'user_info'
port = '3306'  #3306

connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db_name}'
engine = create_engine(connection_string)

In [3]:
sql_query = 'SELECT * FROM user_like;'
user_like = pd.read_sql_query(sql_query, engine)

In [4]:
user_like.set_index("uID",inplace=True)

In [5]:
print(user_like.iloc[:5,:10])

      飲料   早餐    麵   中式    飯   炸物   便當   甜點   蛋餅   鍋物
uID                                                  
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5  0.0  0.0
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5
5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0


In [6]:
sql_query = 'SELECT * FROM new_rlabel;'
res_like = pd.read_sql_query(sql_query, engine)

In [7]:
res_like.set_index("rID",inplace=True)

In [8]:
print(res_like.iloc[:5,:10])

     飲料  早餐  麵  中式  飯  炸物  便當  甜點  蛋餅  鍋物
rID                                      
4     0   0  0   1  0   0   0   0   0   0
5     0   0  0   0  0   0   0   0   0   1
6     0   0  0   0  1   0   0   0   0   0
8     0   0  1   1  0   0   0   0   0   0
9     0   0  0   0  0   0   0   0   0   0


In [9]:
#用餘弦算相似度
user_res_like_similarity_matrix = cosine_similarity(user_like.values,res_like.values)
user_res_like_similarity_matrix = pd.DataFrame(user_res_like_similarity_matrix, index=user_like.index,columns=res_like.index)

num_user,num_res =  user_res_like_similarity_matrix.shape

In [10]:
print(f"user_res_like_similarity_matrix - #user: {num_user}, #res: {num_res}\n")
print(user_res_like_similarity_matrix.iloc[:5,:5])

user_res_like_similarity_matrix - #user: 20, #res: 1440

rID    4         5    6    8    9
uID                              
1    0.0  0.000000  0.0  0.0  0.0
2    0.0  0.149071  0.0  0.0  0.0
3    0.0  0.000000  0.0  0.0  0.0
4    0.0  0.125988  0.0  0.0  0.0
5    0.0  0.000000  0.0  0.0  0.0


In [11]:
def get_the_most_similar_res(user_id, user_res_matrix,num):
    """Find the top-n restaurant most similar to the user"""
    user_like = user_res_matrix.loc[user_id].values 
    sorted_index = np.argsort(user_like)[::-1][:num]
    return list(user_res_matrix.columns[sorted_index])

In [12]:
#restaurant name
sql_query = 'SELECT rID,rName FROM restaurant;'
res_name = pd.read_sql_query(sql_query, engine)

In [13]:
sql_query = 'SELECT rID,rName,all_label FROM res_info;'
res_info = pd.read_sql_query(sql_query, engine)

In [14]:
#Find the top-10 restaurant most similar to the user1
user = 1
res_ids = get_the_most_similar_res(user, user_res_like_similarity_matrix,10)
print(f"user: {user}\n")
print(res_info[res_info["rID"].isin(res_ids)]["rName"])

user: 1

234                                        有利肉圓
239                                  江媽媽筒仔米糕菜頭粿
257                                      杜拉拉小食堂
271                                        佳味麵館
277                                         卓肉丸
312                            果丘in 手作白木耳吸凍飲專賣店
335                                        阿甲肉丸
347                                        阿開肉圓
474                                  埔里雙美紹興米糕老店
493    真好味烤鴨莊-外帶外送團購午晚餐 隱藏排隊美食 拜拜三牲烤雞滷味(在地人氣必吃)
Name: rName, dtype: object


In [15]:
#Find the top-10 restaurant most similar to the user1
user = 2
res_ids = get_the_most_similar_res(user, user_res_like_similarity_matrix,10)
print(f"user: {user}\n")
print(res_info[res_info["rID"].isin(res_ids)]["rName"])

user: 2

102                         小青蛙咖啡吧/埔里美食/咖啡/餐廳/coffee/下午茶
131                                        六扇門時尚湯鍋-埔里八德店
194                                           石二鍋 埔里家樂福店
266                                               亞美石頭火鍋
343                                              阿桐師港式燒臘
401    食午弎 火鍋外送專賣(暫無內用)- 埔里 鍋物 美食 餐廳 午餐 晚餐 推薦 打卡 生日佈置...
417                                              埔里 Mr.鍋
493             真好味烤鴨莊-外帶外送團購午晚餐 隱藏排隊美食 拜拜三牲烤雞滷味(在地人氣必吃)
616                                              鼎上精緻涮涮鍋
692           擼串舌忝籤火鍋串串香/埔里美食/埔里必吃/埔里麻辣火鍋/埔里餐廳/埔里晚餐/埔里火鍋
Name: rName, dtype: object


In [16]:
#Find the top-10 restaurant most similar to the user1
user = 10
res_ids = get_the_most_similar_res(user, user_res_like_similarity_matrix,10)
print(f"user: {user}\n")
print(res_info[res_info["rID"].isin(res_ids)]["rName"])

user: 10

129                                                公道豬排
168                                            台灣第一家鹹酥雞
169                                         台灣第一家鹹酥雞甜不辣
170                                            台灣第一家鹽酥雞
176                                        巧味園脆皮炸雞-埔里總店
185    正好吃-龍鹽酥雞-鎮寶店-埔里鹽酥雞推薦|必吃鹽酥雞|炸物推薦|熱門炸物|推薦小吃|在地推薦美食
439                                             埔里秉峰鍋燒店
599                                               愛蘭香雞排
614                                             詹媽媽脆皮炸雞
718                                               鹽酥雞-沈
Name: rName, dtype: object
